In [11]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dtaidistance import dtw
import itertools
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

import os
import random

from SVM_models.models import evaluate_binary_SVM_standard, evaluate_binary_SVM_custom

# import GunPoint dataset from GunPoint/GunPoint_test.txt and GunPoint/GunPoint_train.txt


def load_dataset(file_path):
    with open(file_path) as f:
        lines = f.readlines()
    # Parse each line, separating the label and features
    labels = []
    features = []
    for line in lines:
        values = list(map(float, line.strip().split()))
        labels.append(int(values[0])-1)  # First value is the label, cast to int
        features.append(values[1:])    # Remaining values are features
    # Convert lists to numpy arrays for easier handling
    labels = np.array(labels)
    features = np.array(features)
    # normalize dataset to 0 1
    features = (features - features.min()) / (features.max() - features.min())
    return labels, features

# Load the GunPoint dataset
train_path = os.path.join("GunPoint", "GunPoint_TRAIN.txt")
test_path = os.path.join("GunPoint", "GunPoint_TEST.txt")

train_labels, train_features = load_dataset(train_path)
INITIAL_TEST_LABELS, INITIAL_TEST_FEATURES = load_dataset(test_path)

# split test in validation (50) and test (100)



In [12]:
EXPS = [
    "RANDOM",
    "RANDOM_BEST",
    "BAYESIAN_HAMILTONIAN",
    "BAYESIAN_MODEL",
]

OPT_TIMES = 30

CONFIG = {
    "N_QUBITS": 2,
    "SPARSITY": 0.95,
    "ENCODING": "euler",
}

SEEDS = [1, 2, 3, 4, 5, 6, 7, 8, 9]
EXP = "RANDOM_BEST"


In [13]:

def save_results(EXP, encoding, seed, train_accuracy, val_accuracy, test_accuracy, Hamiltonian_c, hidden_alphas):
    # Create folder structure
    folder_name = f"{EXP}__{encoding}"
    seed_folder = os.path.join("exps_results", folder_name, f"exp_{seed}")
    os.makedirs(seed_folder, exist_ok=True)

    # Save configuration and OPT_TIMES to a text file
    with open(os.path.join(seed_folder, "config.txt"), "w") as f:
        f.write(f"Experiment: {EXP}\n")
        f.write(f"Encoding: {encoding}\n")
        f.write(f"Number of Qubits: {CONFIG['N_QUBITS']}\n")
        f.write(f"Sparsity Coefficient: {CONFIG['SPARSITY']}\n")
        f.write(f"OPT_TIMES: {OPT_TIMES}\n")

    # Save train, validation, and test accuracy
    with open(os.path.join(seed_folder, "train_accuracy.txt"), "w") as f:
        f.write(f"Train Accuracy: {train_accuracy}\n")
    
    with open(os.path.join(seed_folder, "val_accuracy.txt"), "w") as f:
        f.write(f"Validation Accuracy: {val_accuracy}\n")

    with open(os.path.join(seed_folder, "test_accuracy.txt"), "w") as f:
        f.write(f"Test Accuracy: {test_accuracy}\n")

    # Save Hamiltonian_c and hidden_alphas
    with open(os.path.join(seed_folder, "Hamiltonian_c.txt"), "w") as f:
        f.write(str(Hamiltonian_c))

    with open(os.path.join(seed_folder, "hidden_alphas.txt"), "w") as f:
        f.write(str(hidden_alphas))


In [14]:
from SVM_models.QuantumSVM import QuantumSVM
from sklearn.metrics import accuracy_score

In [15]:
from copy import deepcopy
from time import sleep

for SEED in SEEDS:
    
    test_labels, val_labels, test_features, val_features = train_test_split(INITIAL_TEST_LABELS, INITIAL_TEST_FEATURES, test_size=0.33, random_state=SEED)
    best_val_acc = 0

    for TRIES in range(OPT_TIMES):
        np.random.seed(SEED*100 + TRIES)
        print(f"Seed: {SEED} - Try: {TRIES}")
        quantum_svm = QuantumSVM(n_qubits=CONFIG["N_QUBITS"], sparsity_coefficient=CONFIG["SPARSITY"], embedding_code=CONFIG["ENCODING"])

        quantum_svm.fit(train_features, train_labels)
        y_train = quantum_svm.predict_on_train()
        train_accuracy = accuracy_score(train_labels, y_train)
        print(f"Train Accuracy: {train_accuracy}")

        y_val = quantum_svm.predict(val_features)
        val_accuracy = accuracy_score(val_labels, y_val)
        print(f"Validation Accuracy: {val_accuracy}")



        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            best_quantum_svm = deepcopy(quantum_svm)
            best_train_accuracy = train_accuracy

    y_test = best_quantum_svm.predict(test_features)
    accuracy = accuracy_score(test_labels, y_test)
    print(f"Test Accuracy: {accuracy}")
    # now I want to save the model Hamiltonian_c and hidden_alphas
    print("Saving model...")
    Hamiltonian_c = best_quantum_svm.Hamiltonian_c
    hidden_alphas = best_quantum_svm.hidden_alphas

    #print("Hamiltonian_c: ", Hamiltonian_c)
    #print("hidden_alphas: ", hidden_alphas)

    save_results(EXP, CONFIG["ENCODING"], SEED, best_train_accuracy, best_val_acc, accuracy, Hamiltonian_c, hidden_alphas)
    sleep(600) # sleep for 15 minutes to avoid overloading the server

Seed: 1 - Try: 0

Generating random coefficients for the Hamiltonian...

Generating random alphas...
Non-zero alphas: 12


Building TRAIN_KERNEL: 100%|██████████| 1225/1225 [01:22<00:00, 14.87it/s]


Train Accuracy: 0.76


Building TEST_KERNEL: 100%|██████████| 2500/2500 [02:44<00:00, 15.15it/s]


Validation Accuracy: 0.78
Seed: 1 - Try: 1

Generating random coefficients for the Hamiltonian...

Generating random alphas...
Non-zero alphas: 6


...

Validation Accuracy: 0.86
Seed: 2 - Try: 22

Generating random coefficients for the Hamiltonian...

Generating random alphas...
Non-zero alphas: 9


Building TRAIN_KERNEL: 100%|██████████| 1225/1225 [01:09<00:00, 17.70it/s]


Train Accuracy: 0.86


Building TEST_KERNEL:  27%|██▋       | 669/2500 [00:37<01:43, 17.65it/s]


KeyboardInterrupt: 